## Assignment 1: MongoDB

This assignment is based on content discussed in Module 2: Introduction to MongoDB

## Learning outcomes

The purpose of this assignment is for learners to be able to:
- Familarize with JSON document syntax
- Understand basic MongoDB CRUD operations
- Understand MongoDB data pipelines to run aggregate queries

In this assignment, you will make use of the sample data provided in Module 2.

This dataset has 3 collections: Employee, Workplace and Address.  You will import this data into your local MongoDB database.

Required imports for this project are given below. Make sure you have all libraries required for this project installed. You may use conda or pip based on your set up.

## Setup Notes

**Please note** These instructions are duplicated here for your reference. You would have encountered them originally in Module 2 Part 2. It is expected that you have already taken the steps to set up and run MongoDB.

We will be using MongoDB Community Edition.  The MongoDB database <b>MUST</b> be installed and running locally before continuing with this notebook.  We will need to install two packages using the Anaconda package manager:

1. [Mongodb](https://www.mongodb.com/) - this package contains the mongodb database
2. [PyMongo](http://api.mongodb.com/python/current/) - this packages contains the python driver that will allow us to communicate with the mongodb database.

#### Install

1. Open a command line terminal and execute the following commad to install mongodb.
```console
conda install -c anaconda mongodb
```
2. Open a command line terminal and execute the following commad to install pymongo packge.
```console
conda install -c anaconda pymongo
```

#### Run Mongodb in Windows
1. MongoDB requires a data directory to store all data. MongoDB’s default data directory path is \data\db. Create this folder using the following commands from a Command Prompt:
```console
md \data\db
```

2. To start MongoDB, run mongod.exe. For example, from the Command Prompt:
```console
"C:\Program Files\MongoDB\Server\3.2\bin\mongod.exe"
```

#### Run Mongodb in Mac
1. MongoDB requires a data directory to store all data. MongoDB’s default data directory path is /data/db. Create this folder using the following commands from a Command Prompt.  Note that we run the command as a super user using the "sudo" command:
```console
sudo mkdir /data/db/
```

2. To start MongoDB, run mongod.exe. For example, from the Command Promp.  Note that we run the command as a super user using the "sudo" command:
```console
sudo mongod --config /usr/local/etc/mongod.conf
```

In [3]:
#required imports
import os
import json
import datetime
import pymongo
import pprint
import pandas as pd
import numpy as np
from pymongo import MongoClient
print('Mongo version', pymongo.__version__)

Mongo version 3.11.4


We first need to connect to our locally running MongoDB database (<b>Make sure your database is running on your machine</b>). We will use the MongoClient to connect to a local 'test' database that is running on port 27017 (this is the default port).

In [4]:
client = MongoClient('localhost', 27017)
db = client.assignment1

After installing necessary modules proceed to import the data into your database.

In [18]:
# Let's delete any existing collections in our database
db.workplace.drop()
db.address.drop()
db.employee.drop()

# Import our files into our three collections
with open('datadb/Employee.json') as f:
    db.employee.insert_many(json.load(f))
with open('datadb/Workplace.json') as f:
    db.workplace.insert_many(json.load(f))
with open('datadb/Address.json') as f:
    db.address.insert_many(json.load(f))

#### Question 1 (10 Marks)

The address collection contains employee from different ages and interests.  Perform a simple query to list all employees that are less than or equal to 50 and like Cooking.

__NOTE:__ the following shows the structure of an Employee document that will help you construct the query.

In [19]:
pprint.pprint(client.assignment1.employee.find_one())

{'_id': '9f39da36-82cc-4353-ab90-d616105fa7c1',
 'address_id': 'b6c0b50a-d0e3-43bf-a2a4-8d4674c2a7e8',
 'age': 40,
 'email': 'ih@ri.ro',
 'firstname': 'Emilie',
 'interests': ['Bowling', 'Cooking', 'Golf', 'Swimming'],
 'lastname': 'Woods',
 'workplace_id': 'a32bf18d-e0e5-48f2-a851-aa49c80f9460'}


In [57]:
cursor = db.employee.find({'$and':
                               [{'age':{'$gt': 50}},
                                {'interests' : 'Cooking'}]},
                          {"firstname", "age", "interests"})
pd.DataFrame(list(cursor))

,_id,firstname,age,interests
0,8243f5a6-3f69-42f3-af11-5486cc2273d4,Clyde,54,"[Bowling, Golf, Dancing, Cooking]"
1,fcb42f1d-75ce-4e93-9407-48d7ccc83a3e,Sally,51,"[Swimming, Fishing, Cooking, Bowling, Dancing,..."
2,24c45b35-fc61-44b6-a896-4b78d6d58a55,Eugene,56,"[Soccer, Fishing, Cycling, Cooking, Bowling]"
3,0194bd7a-2b2c-4d7e-8c4a-0b3a97a7bbf0,Ernest,79,"[Rubgy, Cooking, Dancing, Soccer]"
4,4ec72815-86f9-42b7-a2cb-61459c12a352,Loretta,54,"[Bowling, Tennis, Cooking, Hiking]"
5,2acaa277-8551-41e0-b90e-5b4d199aca3f,Henry,59,"[Hiking, Golf, Tennis, Swimming, Fishing, Cook..."
6,7d70194f-9282-4360-b490-ff70e0b03bbf,Lucinda,61,"[Hiking, Golf, Cooking, Fishing]"
7,9ebcf9fc-e193-4ab7-9c38-10c2d6bf6036,Alan,76,"[Bowling, Cooking, Rubgy, Fishing, Golf, Cricket]"
8,a85cdc3c-7279-4461-9556-49373143f41e,Alma,77,"[Cooking, Cycling, Hiking]"
9,c6d0eebd-ddf2-41ed-a20f-76274c5c8343,Tony,72,"[Cooking, Rubgy]"


#### Question 2  (10 Marks)

Insert a new Employee with the following properties:

* First Name: Jake
* Last Name: Sample
* Email: jakesample@email.com
* Age: 26
* Interest: Biking, Hiking

Also, this employee works for 'Union Planters Corp' and lives at '573 Wojhas Square, Victoria'.
Verify that the insert succeeded and display the generated employees _id attribute.

__HINT__ An Employee document references a Workplace and Address document

In [23]:
workplace_jake = db.workplace.find_one({'name': 'Union Planters Corp'}, {'_id'})
address_jake = db.address.find_one({'address': '573 Wojhas Square'}, {'_id'})

db.employee.insert_one({
    'age': '26',
    'email': 'jakesample@email.com',
    'firstname': 'Jake',
    'interests': ['Biking', 'Hiking'],
    'lastname': 'Sample',
    'address_id': str(address_jake['_id']),
    'workplace_id': str(workplace_jake['_id'])
})

cursor = db.employee.find_one({'firstname': 'Jake'}, {'_id'})
pprint.pprint(str(cursor['_id']))

'60b95459b0a158fa59c46b7d'


#### Question 3 (10 Marks)

Delete all employees that work for 'Great Plains Energy Inc' and are greater than 46 years old and likes 'Tennis'. Once you delete the employees verify the number of employees deleted.

In [33]:
great_plains_id = db.workplace.find_one({'name': 'Great Plains Energy Inc.'}, {'_id'})

delete_result = db.employee.delete_many({'$and':
                                             [
                                                 {'workplace_id': str(great_plains_id['_id'])},
                                                 {'age':{'$gt': 46}},
                                                 {'interests' : 'Tennis'}
                                             ]
})
print(delete_result.acknowledged)
print(delete_result.deleted_count)

True
4


#### Question 4 (12 Marks)
Add a new field called 'industry' to all employees that work for 'Health Net Inc.'.

__HINT__ All a new field to a document is like updating the document

In [38]:
health_net_id = db.workplace.find_one({"name": "Health Net Inc."}, {"_id"})
db.employee.update_many({"workplace_id": str(health_net_id['_id'])},
                        {"$set": {"industry": "NEW_FIELD"}})

cursor = db.employee.find({"workplace_id" : str(health_net_id['_id'])},
                          {"firstname", "workplace_id", "industry"})
pd.DataFrame(list(cursor))

,_id,firstname,workplace_id,industry
0,ad4fd8bd-b732-4388-8469-5395b497df63,Marc,a222385c-342c-43ea-adbc-9b487a2ee2be,NEW_FIELD
1,6222b4f5-6fd3-4c72-b058-710fb568b869,Lucinda,a222385c-342c-43ea-adbc-9b487a2ee2be,NEW_FIELD
2,ca189ae9-d04b-4ae0-9b4a-b725196c31d2,Emilie,a222385c-342c-43ea-adbc-9b487a2ee2be,NEW_FIELD
3,ed499274-f17f-4a4c-8327-7e740e6dd9c1,Josephine,a222385c-342c-43ea-adbc-9b487a2ee2be,NEW_FIELD
4,6a3bd505-56e6-46ec-bef1-2eef1fd3643c,Vernon,a222385c-342c-43ea-adbc-9b487a2ee2be,NEW_FIELD
5,5b8bb404-5741-46a9-84f2-81248ffd2840,Glen,a222385c-342c-43ea-adbc-9b487a2ee2be,NEW_FIELD
6,c7cf8650-f391-4f2b-bbb5-60225729e998,Jeremiah,a222385c-342c-43ea-adbc-9b487a2ee2be,NEW_FIELD
7,9ebcf9fc-e193-4ab7-9c38-10c2d6bf6036,Alan,a222385c-342c-43ea-adbc-9b487a2ee2be,NEW_FIELD
8,fd677b4b-6eef-4777-bc3a-25d9a86134fa,Mattie,a222385c-342c-43ea-adbc-9b487a2ee2be,NEW_FIELD
9,cb369c19-4bbc-449f-bff6-b198439d0d7b,Adele,a222385c-342c-43ea-adbc-9b487a2ee2be,NEW_FIELD


#### Question 5 (10 Marks)

Create an aggregate query to count the number of employees for each company and sort the output from largest employee count to lowest employee count.

__NOTE__ you will use a pipeline to achieve the computed result.  You should produce a result similar to the following table (the following table contains fake data)
<table>
    <tr><th></th><th>_id</th><th>count</th></tr>
    <tr><td>0</td><td>[Equity Residential Properties Trust]</td><td>19</td></tr>
    <tr><td>...</td><td>...</td><td>...</td></tr>
    <tr><td>7</td><td>[Bell Microproducts Inc.]</td><td>6</td></tr>
    <tr><td>8</td><td>[Kemet Corp.]</td><td>1</td></tr>
</table>

__HINT__ you should make use of the \\$lookup, \\$group and \\$sort pipeline operations

In [56]:
pipeline = [
    {'$lookup':
        {
            "from": "workplace",
            "localField": "workplace_id",
            "foreignField": "_id",
            "as": "workplace"
        }},
    {'$group':
            {"_id": "$workplace.name",
             "count": {"$sum": 1}}
     },
    {"$sort": {"count": -1}}
]

query_result = db.employee.aggregate(pipeline)

pd.DataFrame(list(query_result))

,_id,count
0,[Hilton Solutions],15
1,[Health Net Inc.],14
2,[Aetna Inc.],13
3,[Bell Microproducts Inc.],11
4,[Union Planters Corp],11
5,[Equity Office Properties Trust],10
6,[Equity Residential Properties Trust],7
7,[Xcel Bear Inc],6
8,[Kemet Corp.],6
9,[Great Plains Energy Inc.],5
